### Matching

In [1]:
import networkx as nx
from scipy.spatial import distance
from tqdm import tqdm
import pandas as pd
import numpy as np

import concurrent.futures

pd.set_option('display.max_columns', None)

In [2]:
# Cols that we want to Match
cols_m = [
    'log Area Comercial',
    'log Area Residencial - V',
    'log Area Residencial - H',
    'log Area Terreno',
    'CA'
]

In [3]:
# Read the data
df_results = pd.read_parquet('df_results_1000.parquet')
df_results.sample(10)

,sq,geometry,Nome,Linha,elegivel,T,Inauguração,ANO DO EXERCICIO,AREA OCUPADA,AREA DO TERRENO,AREA CONSTRUIDA,Area Comercial,Area Residencial - H,Area Residencial - V,Area Terreno,Residencial - H,Residencial - V,Comercial,Terreno,QUANTIDADE DE PAVIMENTOS,VALOR DO M2 DE CONSTRUCAO,VALOR DO M2 DO TERRENO,Valor,ANO DE INICIO DA VIDA DO CONTRIBUINTE,CA,TO,ano,delta,per Area Comercial,per Area Residencial - V,per Area Residencial - H,diff AREA CONSTRUIDA,diff Area Comercial,diff Area Residencial - H,diff Area Residencial - V,sum AREA CONSTRUIDA,sum Area Comercial,sum Area Residencial - H,sum Area Residencial - V,log AREA CONSTRUIDA,log Area Comercial,log Area Residencial - H,log Area Residencial - V,log Area Terreno
524009,166255,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00'\x00\x0...",Campo Limpo,5,1,0,2002-10-20,2011,636,1903,825,0,824,0,0,1.000000,0.000000,0.000000,0.000000,1.499394,484.200000,52.000000,0.423726,1985.278788,0.433526,0.334209,2002,9,0.000000,0.000000,0.998788,0.0,0.0,0.0,0.0,241.0,0.0,241.0,0.0,6.715383,0.000000,6.714171,0.000000,0.000000
130480,152035,b'\x01\x03\x00\x00\x00\x01\x00\x00\x003\x00\x0...,Sapopemba,15,1,1,2019-12-16,2000,2236,7572,4142,3072,1069,0,446,0.600000,0.000000,0.266667,0.133333,1.217045,174.933333,20.533333,0.818737,1971.682762,0.547015,0.295298,2019,-19,0.741671,0.000000,0.258088,0.0,0.0,0.0,0.0,143.0,0.0,143.0,0.0,8.328934,8.030084,6.974479,0.000000,6.100319
106831,118247,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00/\x00\x0...",São Lucas,15,1,1,2018-06-04,2013,3035,4886,3830,340,3489,0,0,0.961538,0.000000,0.038462,0.000000,1.731593,700.115385,285.384615,2.729490,1975.188251,0.783872,0.621163,2018,-5,0.088773,0.000000,0.910966,0.0,0.0,0.0,0.0,697.0,120.0,577.0,0.0,8.250620,5.828946,8.157370,0.000000,0.000000
372023,116364,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00I\x00\x0...,Jardim Planalto,15,1,0,2019-08-26,2005,8871,20051,11622,2235,7069,0,600,0.736842,0.000000,0.175439,0.017544,1.585441,409.368421,170.000000,5.221242,1971.797281,0.579622,0.442422,2019,-14,0.192308,0.000000,0.608243,-19.0,0.0,-19.0,0.0,2518.0,255.0,883.0,0.0,9.360655,7.711997,8.863474,0.000000,6.396930
281570,051204,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00[\x00\x0...,Oratório,15,1,0,2014-08-30,2012,6181,10500,8568,1905,6662,0,150,0.865385,0.000000,0.115385,0.019231,1.688025,603.596154,371.076923,5.504556,1968.965336,0.816000,0.588667,2014,-2,0.222339,0.000000,0.777544,0.0,0.0,0.0,0.0,-191.0,166.0,-357.0,0.0,9.055790,7.552237,8.804175,0.000000,5.010635
3866,012073,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00V\x00\x0...,Vila Madalena,2,1,1,1998-11-21,2017,9913,26283,53197,1609,2824,41739,0,0.609756,0.219512,0.048780,0.000000,17.181815,1644.852315,2140.512195,105.576058,1988.638570,2.024008,0.377164,1998,19,0.030246,0.784612,0.053086,0.0,0.0,0.0,0.0,25389.0,1412.0,-2305.0,19387.0,10.881757,7.383368,7.945910,10.639191,0.000000
222419,089590,b'\x01\x03\x00\x00\x00\x01\x00\x00\x007\x00\x0...,Campo Belo,5,1,0,2019-08-04,2003,2508,4615,3591,193,3397,0,0,0.967742,0.000000,0.032258,0.000000,1.794486,364.580645,305.096774,1.295494,1983.447508,0.778115,0.543445,2019,-16,0.053745,0.000000,0.945976,-30.0,-118.0,88.0,0.0,68.0,0.0,68.0,0.0,8.186186,5.262690,8.130648,0.000000,0.000000
178459,084071,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00X\x00\x0...,Eucaliptos,5,1,0,2018-02-03,2004,3577,10205,5302,0,5301,0,532,0.888889,0.000000,0.000000,0.111111,1.865522,593.777778,407.000000,3.541068,1976.784044,0.519549,0.350514,2018,-14,0.000000,0.000000,0.999811,0.0,0.0,0.0,0.0,32.0,0.0,32.0,0.0,8.575839,0.000000,8.575651,0.000000,6.276643
476744,185137,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00O\x00\x0...,Vila Sônia,4,1,0,2019-12-17,2010,3074,6237,3865,0,3724,0,0,0.975000,0.000000,0.000000,0.000000,1.512807,483.125000,141.475000,1.940460,1972.551100,0.619689,0.492865,2019,-9,0.000000,0.000000,0.963519,0.0,0.0,0.0,0.0,445.0,0.0,445.0,0.0,8.259717,0.000000,8.222554,0.000000,0.000000
40233,041222,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00)\x00\x0...,Eucaliptos,5,1,1,2018-02-03,2008,6

In [4]:
def matching_algo(treated, control):
    # Create an empty graph
    graph = nx.Graph()

    # Add treated units to the graph as nodes
    for _, treated_row in treated.iterrows():
        graph.add_node(treated_row.sq, bipartite=0)

    # Add control units to the graph as nodes
    for _, control_row in control.iterrows():
        graph.add_node(control_row.sq, bipartite=1)

    # We iterate: for each treated and for each control unitm
    # We calculate the distance in respespect over the macthing columns
    # The distante is the weight edge
    for _, treated_row in treated.iterrows():
        for _, control_row in control.iterrows():
            dist = distance.euclidean(treated_row[cols_m], control_row[cols_m])
            graph.add_edge(treated_row.sq, control_row.sq, weight=dist)
    
    # So we calculate the distance for each pair
    # So now we apply the algorithm
    # It'll return the perfect match which minimize the distance
    matching = nx.bipartite.minimum_weight_full_matching(graph)
    return matching

def finding_matching(df, nome):
    # Select the ids columns + Matching columns
    cols = ['Nome', 'sq', 'T'] + cols_m
    
    # Filter only the data we need
    # Group we want and the prior
    df_f = df_results[
        (df.delta == -3) 
        & (df.Nome == nome)
    ][cols]
    
    # Separate treated and control units    
    treated = df_f[df_f['T'] == 1]
    control = df_f[df_f['T'] == 0]
    
    # Apply Matching algorithm
    matching = matching_algo(treated, control)
    
    # Return DataFrame
    df_matching = pd.concat([
        control[control.sq.isin([matching[e] for e in treated.sq])],
        treated
    ])
    return df_matching

In [5]:
# Selecting all the Metro Stations Groups to iterate over
aux_list_result = set(df_results.Nome.unique())

# This process takes timing
# To avoid that, we use a multiprocessing.
# That means, instead of iterating one at time, we have 8 workers
with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
    # Apply algorithm
    futures = [executor.submit(finding_matching, df_results, i) for i in aux_list_result]
    # Collecting the resulting and tracking the processing
    results = [future.result() for future in tqdm(concurrent.futures.as_completed(futures))]

29it [07:04, 14.63s/it]


In [6]:
# Concat the results into a unified dataframe
df_results_matching = pd.concat(results)

In [7]:
# Lets see before the matching
pd.DataFrame(df_results[df_results.delta == -3].groupby(['T'])[cols_m].mean())

,log Area Comercial,log Area Residencial - V,log Area Residencial - H,log Area Terreno,CA
T,,,,,
0,4.572859,1.436829,7.099200,3.839713,0.848378
1,5.680826,2.295123,7.267711,2.884206,1.054524


In [8]:
# Apply an inner join to select only the matched units
df_results_matching_all = pd.merge(
    df_results,
    df_results_matching[['Nome', 'sq', 'T']],
    on = ['Nome', 'sq', 'T']
)
# Let's compare after matching
df_results_matching_all[df_results_matching_all.delta == -3].groupby(['T'])[cols_m].mean()

,log Area Comercial,log Area Residencial - V,log Area Residencial - H,log Area Terreno,CA
T,,,,,
0,5.736559,1.941957,7.280266,2.963943,1.017246
1,5.680826,2.295123,7.267711,2.884206,1.054524


In [9]:
df_results_matching_all.to_parquet('did_matching_1000.parquet')